In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import cv2
import datetime
import math
import numpy as np
import os
import pathlib
import skimage.io
import skimage.transform
import sklearn.model_selection
import sys
import tensorflow as tf
import time

from albumentations import (
    Compose, HorizontalFlip, ShiftScaleRotate, ElasticTransform,
    RandomBrightness, RandomContrast, RandomGamma
)

from AugmentationSequence import AugmentationSequence
from files import create_folder, save_fit_history, save_lossgraph, save_figs
from metrics import dice_coef, jaccard_distance
from model import evaluate, unet_model, get_loss_function
from save import save

2022-10-10 01:20:36.824274: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-10 01:20:36.949412: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-10 01:20:37.439927: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/xandao/miniconda3/lib/python3.8/site-packages/cv2/../../lib64:
2022-10-10 01:20:37.439983: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.s

In [2]:
class CreateSequence(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return batch_x, batch_y

# GPU

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            print(f'GPU: {tf.config.experimental.get_device_details(gpu)["device_name"]}')
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

GPU: NVIDIA GeForce RTX 3060


2022-10-10 01:20:38.085154: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-10 01:20:38.104958: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-10 01:20:38.105117: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-10 01:20:38.105844: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
cfg = {
    'channel': 3,
    'batch_size': 4,
    'fold': 2,
    'epochs': 10,
    'image_size': 512,
    'learning_rate': 0.001,
    'random_state': 1234,
    'test_size': 0.2,
    'val_size': 0.05,
    'path_dataset': '../dataset',
    'path_out': 'out',
    'loss_function': 'dice',
    'data_augmentation': False
}
# images_folder = os.path.join(cfg['path_dataset'], 'imgs', 'CONVERTIDAS', 'RGB', '256', 'OUT')
# masks_folder = os.path.join(cfg['path_dataset'], 'new_mask', 'BITMAP', '256', 'OUT')
images_folder = "img"
masks_folder = "mask"

In [5]:
list_labels = list([])
list_images = list([])
list_images_names = list([])
def load_files():
    for file in sorted(pathlib.Path(masks_folder).rglob('*')):
        mask = skimage.io.imread(file.resolve())
        mask = np.float32(mask/255)
        list_labels.append(mask)

        image = skimage.io.imread(os.path.join(images_folder, f'{file.stem}.jpeg'))
        image = np.float32(image/255)
        list_images.append(image)

        list_images_names.append(str(file.stem))

load_files()
print(len(list_labels), len(list_images), len(list_images_names))

1 1 1


In [6]:
a = skimage.io.imread("mask/ALCB002266.bmp")
# list_labels[0].shape
a.shape

(1024, 777)

In [8]:
x = np.array(list_images).reshape((len(list_images), cfg['image_size'], cfg['image_size'], cfg['channel']))
y = np.array(list_labels).reshape((len(list_labels), cfg['image_size'], cfg['image_size'], 1))

print(x.shape, y.shape)

ValueError: cannot reshape array of size 795648 into shape (1,512,512,1)

In [ ]:
kf = sklearn.model_selection.KFold(n_splits=cfg['fold'], shuffle=True, random_state=cfg['random_state'])

models = []
list_evaluate = list([])
list_time = 0
current_datetime = datetime.datetime.now().strftime('%d-%m-%Y-%H-%M-%S')
path = os.path.join(cfg['path_out'], current_datetime)
create_folder(list([path]))
for fold, (train_index, test_index) in enumerate(kf.split(x)):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(x_train, y_train, test_size=cfg['val_size'], random_state=cfg['random_state'])

    print(x_train.shape)
    print(x_val.shape)
    print(x_test.shape)
    print(x.shape)

    path_fold = os.path.join(path, str(fold))
    create_folder(list([path_fold]))

    augment = Compose([
        HorizontalFlip(),
        ShiftScaleRotate(rotate_limit=45, border_mode=cv2.BORDER_CONSTANT),
        ElasticTransform(border_mode=cv2.BORDER_CONSTANT),
        RandomBrightness(),
        RandomContrast(),
        RandomGamma()
    ])
    steps_per_epoch = math.ceil(x_train.shape[0] / cfg['batch_size'])
    if cfg['data_augmentation']:
        train_generator = AugmentationSequence(x_train, y_train, cfg['batch_size'], augment)
    else:
        train_generator = CreateSequence(x_train, y_train, cfg['batch_size'])
    reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose=1)
    filename_model = os.path.join(path_fold, 'unet.h5')
    checkpointer = tf.keras.callbacks.ModelCheckpoint(filename_model, verbose=1, save_best_only=True)
    strategy = tf.distribute.MirroredStrategy()

    with strategy.scope():
        model = unet_model(cfg)
        adam_opt = tf.keras.optimizers.Adam(learning_rate=cfg['learning_rate'])
        model.compile(optimizer=adam_opt, loss=get_loss_function(cfg['loss_function']), metrics=[dice_coef, jaccard_distance, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    tf.keras.backend.clear_session()
    start_time = time.time()
    fit = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=cfg['epochs'], validation_data=(x_val, y_val), callbacks=[checkpointer, reduce_learning_rate])
    end_time = time.time() - start_time

    save_fit_history(fold, fit, path_fold)
    save_lossgraph(fold, fit, path_fold)
    list_evaluate.append(evaluate(end_time, fold, model, x_train, x_val, x_test, y_train, y_val, y_test))
    list_time += end_time

    models.append(model)

    # model = tensorflow.keras.models.load_model('unet_rgb.h5', custom_objects = {'dice_loss': dice_loss, 'dice_coef': dice_coef, 'jaccard_distance': jaccard_distance })

    save_figs(cfg, list_images_names, test_index, model, path_fold, x)

tf.keras.backend.clear_session()

In [ ]:
save(cfg, sys.argv, images_folder, list_evaluate, list_images, list_labels, list_time, masks_folder, path)